# Computer Vision Analysis
#### By Ronny Toribio, Kadir O. Altunel, Michael Cook-Stahl
#### Based on [Hands on Machine Learning 2nd edition](https://github.com/ageron/handson-ml2/), [FER2013 candidate 1](https://www.kaggle.com/code/ritikjain00/model-training-fer-13) and [FER2013 candidate 2](https://www.kaggle.com/code/gauravsharma99/facial-emotion-recognition/notebook)

### Import modules and declare constants

In [ ]:
%matplotlib inline
import os.path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import precision_recall_curve
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense, BatchNormalization,
     LeakyReLU
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger, ModelCheckpoint 
from google.colab import drive

### Constants

In [ ]:
BATCH_SIZE = 32
IMAGE_ZOOM = 0.3
IMAGE_SHAPE = (48, 48)
INPUT_SHAPE = (48, 48, 1)
TRAIN_DIR = os.path.join("fer2013", "train")
TEST_DIR = os.path.join("fer2013", "test")

### Google Drive

In [ ]:
drive.mount('/content/drive')
!unzip /content/drive/MyDrive/FER.zip

### Load Facial Emotion Recognition dataset
#### training, validation, and testing

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=IMAGE_ZOOM, horizontal_flip=True,
                                   validation_split=0.10)
Xy_train = train_datagen.flow_from_directory(TRAIN_DIR, batch_size=BATCH_SIZE, 
                                   target_size=IMAGE_SHAPE, shuffle=True, subset="training",
                                   color_mode="grayscale", class_mode="categorical")

Xy_valid = train_datagen.flow_from_directory(TRAIN_DIR, batch_size=BATCH_SIZE, 
                                   target_size=IMAGE_SHAPE, shuffle=True, subset="validation",
                                   color_mode="grayscale", class_mode="categorical")

test_datagen = ImageDataGenerator(rescale=1./255)
Xy_test = test_datagen.flow_from_directory(TEST_DIR, batch_size=BATCH_SIZE * 1000,
                                   target_size=IMAGE_SHAPE, shuffle=True,
                                   color_mode="grayscale", class_mode="categorical")

y_test = Xy_test[0][1]

### Build and compile CNN model

In [ ]:
def build_model(main_activation, main_initializer, use_conv_block3=False):
    model = Sequential(name="cnn_model")
    # Convolution Block 0
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, input_shape=INPUT_SHAPE,
              name="conv_block_0_conv_layer_0_input"))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, name="conv_block_0_conv_layer_1"))
    model.add(BatchNormalization(name="conv_block_0_batchnorm"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid", name="conv_block_0_maxpool"))
    model.add(Dropout(0.25, name="conv_block_0_dropout"))

    # Convolution Block 1
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, kernel_regularizer=l2(0.01), name="conv_block_1_conv_layer_0"))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, kernel_regularizer=l2(0.01), name="conv_block_1_conv_layer_1"))
    model.add(BatchNormalization(name="conv_block_1_batchnorm"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="valid", name="conv_block_1_maxpool"))
    model.add(Dropout(0.25, name="conv_block_1_dropout"))
        
    # Convolution Block 2
    model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, kernel_regularizer=l2(0.01),
              name="conv_block_2_conv_layer_0"))
    model.add(Conv2D(filters=1024, kernel_size=(3, 3), padding="same", activation=main_activation,
              kernel_initializer= main_initializer, kernel_regularizer=l2(0.01), 
              name="conv_block_2_conv_layer_1"))
    model.add(BatchNormalization(name="conv_block_2_batchnorm"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="valid", name="conv_block_2_maxpool"))
    model.add(Dropout(0.25, name="conv_block_2_dropout"))
    
    # Convolution Block 3
    if use_conv_block3:
        model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", activation=main_activation,
                  kernel_initializer= main_initializer, kernel_regularizer=l2(0.01),
                  name="conv_block_3_conv_layer_0"))
        model.add(Conv2D(filters=1024, kernel_size=(3, 3), padding="same", activation=main_activation,
                  kernel_initializer= main_initializer, kernel_regularizer=l2(0.01), 
                  name="conv_block_3_conv_layer_1"))
        model.add(BatchNormalization(name="conv_block_3_batchnorm"))
        model.add(MaxPooling2D(pool_size=(2, 2), padding="valid", name="conv_block_3_maxpool"))
        model.add(Dropout(0.25, name="conv_block_3_dropout"))

    # Classification Block
    model.add(Flatten(name="class_block_flatten"))
    model.add(Dense(1024, activation=main_activation, kernel_initializer="he_normal",
              name="class_block_flatten_dense_0"))
    model.add(BatchNormalization(name="class_block_batchnorm"))
    model.add(Dropout(0.5, name="class_block_dropout"))
    model.add(Dense(7, activation="softmax", name="class_block_dense_output"))
    model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001, decay=1e-6),
              metrics=["accuracy"])
    return model

def build_model_with_activation_layers(main_activation_layer, main_activation_name, use_conv_block3=False):
    model = Sequential(name="cnn_model")
    # Convolution Block 0
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding="same", input_shape=INPUT_SHAPE,
              name="conv_block_0_conv_layer_0_input"))
    model.add(main_activation_layer, name="conv_block_0_{}_0".format(main_activation_name))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding="same", name="conv_block_0_conv_layer_1"))
    model.add(main_activation_layer, name="conv_block_0_{}_1".format(main_activation_name))
    model.add(BatchNormalization(name="conv_block_0_batchnorm"))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding="valid", name="conv_block_0_maxpool"))
    model.add(Dropout(0.25, name="conv_block_0_dropout"))

    # Convolution Block 1
    model.add(Conv2D(filters=128, kernel_size=(3, 3), padding="same", kernel_regularizer=l2(0.01),
              name="conv_block_1_conv_layer_0"))
    model.add(main_activation_layer, name="conv_block_1_{}_0".format(main_activation_name))
    model.add(Conv2D(filters=256, kernel_size=(3, 3), padding="same", kernel_regularizer=l2(0.01),
              name="conv_block_1_conv_layer_1"))
    model.add(main_activation_layer, name="conv_block_1_{}_1".format(main_activation_name))
    model.add(BatchNormalization(name="conv_block_1_batchnorm"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="valid", name="conv_block_1_maxpool"))
    model.add(Dropout(0.25, name="conv_block_1_dropout"))
        
    # Convolution Block 2
    model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", kernel_regularizer=l2(0.01),
              name="conv_block_2_conv_layer_0"))
    model.add(main_activation_layer, name="conv_block_2_{}_0".format(main_activation_name))
    model.add(Conv2D(filters=1024, kernel_size=(3, 3), padding="same", kernel_regularizer=l2(0.01), 
              name="conv_block_2_conv_layer_1"))
    model.add(main_activation_layer, name="conv_block_2_{}_1".format(main_activation_name))
    model.add(BatchNormalization(name="conv_block_2_batchnorm"))
    model.add(MaxPooling2D(pool_size=(2, 2), padding="valid", name="conv_block_2_maxpool"))
    model.add(Dropout(0.25, name="conv_block_2_dropout"))
    
    # Convolution Block 3
    if use_conv_block3:
        model.add(Conv2D(filters=512, kernel_size=(3, 3), padding="same", kernel_regularizer=l2(0.01),
                  name="conv_block_3_conv_layer_0"))
        model.add(main_activation_layer, name="conv_block_3_{}_0".format(main_activation_name))
        model.add(Conv2D(filters=1024, kernel_size=(3, 3), padding="same", kernel_regularizer=l2(0.01), 
                  name="conv_block_3_conv_layer_1"))
        model.add(main_activation_layer, name="conv_block_3_{}_1".format(main_activation_name))
        model.add(BatchNormalization(name="conv_block_3_batchnorm"))
        model.add(MaxPooling2D(pool_size=(2, 2), padding="valid", name="conv_block_3_maxpool"))
        model.add(Dropout(0.25, name="conv_block_3_dropout"))

    # Classification Block
    model.add(Flatten(name="class_block_flatten"))
    model.add(Dense(1024, name="class_block_flatten_dense_0"))
    model.add(main_activation_layer, name="class_block_{}_0".format(main_activation_name))
    model.add(BatchNormalization(name="class_block_batchnorm"))
    model.add(Dropout(0.5, name="class_block_dropout"))
    model.add(Dense(7, activation="softmax", name="class_block_dense_output"))
    model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.0001, decay=1e-6),
              metrics=["accuracy"])
    return model



### Early stopping callback

In [ ]:
early_stopping_cb = EarlyStopping(min_delta=0.00005, patience=11, verbose=1, restore_best_weights=True)

### Reduce learning rate on plateau callback

In [ ]:
reduce_lr_cb = ReduceLROnPlateau(factor=0.5, patience=7, min_l=1e-7, verbose=1)

### Grid Search Hyperparameters for our model

In [ ]:
param_grid = [
    {"main_activation": ["relu"], "main_initializer":["he_normal"], "use_conv_block3": [True, False]},
    {"main_activation_layer": [LeakyReLU(kernel_initializer="he_normal")], "main_activation_name": ["leaky_relu"],
     "use_conv_block3": [True, False]}
]
model_scores = {}
best_model = None
best_model_score = None
best_model_history = None
best_model_name = ""

for params in list(ParameterGrid(param_grid)):
    cur_name = ""
    cur_model = None
    if "main_activation" in params:
        cur_name = "{}+{}+{}".format(params["main_activation_name"], params["main_initializer"],
                                     params["use_conv_block3"])
        cur_model = build_model(**params)
    else:
        cur_name = "{}+{}".format(params["main_activation_name"], params["use_conv_block3"])
        cur_model = build_model_with_activation_layers(**params)
    print(cur_name)
    if cur_model is None:
        continue
    model_json = cur_model.to_json()
    with open(cur_name + ".json", "w") as f:
        f.write(model_json)
    csv_cb = CSVLogger(cur_name + "-training.txt")
    cur_weights = cur_name + "-weights.h5"
    checkpoint_cb = ModelCheckpoint(cur_weights, monitor = 'accuracy', verbose =1, 
                                    save_best_only = True, save_weights_only = True)
    history = cur_model.fit(Xy_train, epochs=80,
                            validation_data=(Xy_valid),
                            steps_per_epoch=Xy_train.n // BATCH_SIZE,
                            validation_steps=Xy_valid.n // BATCH_SIZE,
                            callbacks=[early_stopping_cb, reduce_lr_cb, 
                                       csv_cb, checkpoint_cb])
    cur_model.load_weights(cur_weights)
    loss, accuracy = cur_model.evaluate(Xy_test)
    print("Best: {} Score: {} Loss: {}".format(cur_name, accuracy, loss))
    model_scores[cur_name] = accuracy
    if best_model_score is None or accuracy > best_model_score:
        best_model_score = accuracy
        best_model = cur_model
        best_model_history = history
        best_model_name = cur_name

print()
print("Best: {} Score: {}".format(best_model_name, best_model_score))